## Read in preprocessed data

In [63]:
import pandas as pd
import numpy as np
import time

np.random.seed(42)

X_train = pd.read_pickle('../data/X_train_v3.pkl')
y_train = pd.read_pickle('../data/y_train_v3.pkl')

_______________________________________________________________________________________________________________________________

## Build Models

In [64]:
#Import cross validation and optimization
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

#Import models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from multiprocessing import cpu_count

In [65]:
#Create a dictionary to hold models
models = {}

In [66]:
#Create function to add model and metrics to dictionary
def model_eval(model, name, X_train, y_train, cv):
    #Fit model
    model.fit(X_train, y_train)
    
    #Create predictions
    y_pred = model.predict(X_train)
    
    #Create cross validation scores
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=cpu_count())
    
    #Create cross validation predictions
    cv_pred = cross_val_predict(model, X_train, y_train, cv=cv, n_jobs=cpu_count())
    
    #Create metrics
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    roc_auc = roc_auc_score(y_train, y_pred)
    
    #Add metrics to dictionary
    models[name] = {'model': model,
                    'accuracy': accuracy,
                    'precision': precision,
                    'recall': recall,
                    'f1': f1,
                    'roc_auc': roc_auc,
                    'cv_score': cv_scores.mean(),
                    'cv_pred': cv_pred
                    }
    


In [67]:
#Instantiate models
rf_clf = RandomForestClassifier()
log_reg = LogisticRegression()
gbc = GradientBoostingClassifier()
svc_clf = SVC()
sgd_clf = SGDClassifier()
neigh_clf = KNeighborsClassifier()
dct_clf = DecisionTreeClassifier()
xgb_clf = XGBClassifier()

In [68]:
#Create a list of models
model_list = [rf_clf, log_reg, gbc, svc_clf, sgd_clf, neigh_clf, dct_clf, xgb_clf]

In [69]:
#iterate through list of models, evaluate, and add to dictionary
for model in model_list:
    model_eval(model, model.__class__.__name__, X_train, y_train, 10)

In [85]:
#For each metric, select the best model
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'cv_score']:
    best_score = 0
    for model in models:
        if models[model][metric] > best_score:
            best_score = models[model][metric]
            best_model = model
    print('Best {}: {}'.format(metric, best_model))

Best accuracy: RandomForestClassifier
Best precision: DecisionTreeClassifier
Best recall: RandomForestClassifier
Best f1: RandomForestClassifier
Best roc_auc: RandomForestClassifier
Best cv_score: GradientBoostingClassifier


In [74]:
#Get confusion matrix for RandomForestClassifier
y_pred = models['RandomForestClassifier']['model'].predict(X_train)
confusion_matrix(y_train, y_pred)

array([[546,   3],
       [ 15, 327]])

In [75]:
#Get models from dict with roc_auc > 0.95
best_models = {}
for model in models:
    if models[model]['roc_auc'] > 0.95:
        best_models[model] = models[model]


In [76]:
#Print model and roc_auc
for model in best_models:
    print(model)
    print('roc_auc: ', best_models[model]['roc_auc'])
    print

RandomForestClassifier
roc_auc:  0.9753379350014381
DecisionTreeClassifier
roc_auc:  0.9747866935097307
XGBClassifier
roc_auc:  0.95872878918608


In [77]:
#Print model and cv_score
for model in models:
    print(model)
    print('cv_score: ', models[model]['cv_score'])
    print

RandomForestClassifier
cv_score:  0.8160049937578027
LogisticRegression
cv_score:  0.8002372034956304
GradientBoostingClassifier
cv_score:  0.8294506866416977
SVC
cv_score:  0.8226966292134831
SGDClassifier
cv_score:  0.7429588014981274
KNeighborsClassifier
cv_score:  0.8069912609238452
DecisionTreeClassifier
cv_score:  0.7923845193508114
XGBClassifier
cv_score:  0.8125967540574284


In [78]:
#Optimize RandomForestClassifier
# start = time.time()
# param_grid = [{'n_estimators': [3, 10, 30, 100, 300, 1000],
#                 'max_features': [2, 4, 6, 8, 10],
#                 'max_depth': [2, 4, 6, 8, 10],
#                 'max_leaf_nodes': [2, 4, 6, 8, 10, 20, 50],
#                 'n_jobs': [-1]}]

# rf_clf = RandomForestClassifier()

# scoring = ['roc_auc', 'accuracy', 'precision', 'recall', 'f1']

# grid_search = GridSearchCV(rf_clf, param_grid, cv=10, 
#                            scoring=scoring,
#                            refit='roc_auc', 
#                            n_jobs=cpu_count())

# grid_search.fit(X_train, y_train)

# best_params = grid_search.best_params_

# stop = time.time()
# total_time = stop - start

# print(best_params)
# print(total_time)


In [79]:
# best_params = best_params
# print(best_params)

In [80]:
# #Fit model with best params
# rf_clf = RandomForestClassifier(**best_params)

# #Fit the model
# rf_clf.fit(X_train, y_train)

# #Make predictions
# y_pred = rf_clf.predict(X_train)

# #Evalutate model
# accuracy = accuracy_score(y_train, y_pred)
# precision = precision_score(y_train, y_pred)
# recall = recall_score(y_train, y_pred)
# f1 = f1_score(y_train, y_pred)
# roc_auc = roc_auc_score(y_train, y_pred)

# #Create dict of metrics
# metrics = {'accuracy': accuracy,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1,
#             'roc_auc': roc_auc}

# print(metrics)